In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext


In [2]:
reviews_path = "./data/reviews_Musical_Instruments.json.gz"
meta_path = "./data/meta_Musical_Instruments.json.gz"


In [17]:
conf = SparkConf()
# sc = SparkContext(conf=conf)


In [4]:
spark = (
    SparkSession.builder.appName("Python Spark SQL basic example")
    .config(conf=conf)
    .getOrCreate()
)

reviews_df = spark.read.json(path=reviews_path).select("asin", "overall")
meta_df = spark.read.json(path=meta_path).select("asin", "price")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/06 20:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/09/06 20:21:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
reviews_rdd = reviews_df.rdd.map(lambda x: (x[0], (1, x[1])))
reviews = reviews_rdd.reduceByKey(
    lambda x, y: tuple([a + b for a, b in zip(x, y)])
).map(lambda x: (x[0], (x[1][0], x[1][1] / x[1][0])))

meta = meta_df.rdd.map(lambda x: (x[0], x[1])).filter(lambda x: x[1] != None)


In [13]:
overall = reviews.join(meta).top(10, key=lambda x: x[1][0][0])


In [22]:
spark.sparkContext.parallelize(overall).repartition(1).saveAsTextFile("out")
